In [1]:
import keras

from models import matting_net
import coremltools

Using TensorFlow backend.
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
h5_path = "./trained_models/Strongmodels/455-0.03.h5"
mlmodel_path = "./mlmodel/SOTA.mlmodel"
model = matting_net((256,256,3), android=False, train=False)
model.load_weights(h5_path, by_name=True)

In [3]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 256, 256, 8)  144         re_lu_1[0][0]                    
____________________________________________________________________________________________

In [4]:
ml_model = coremltools.converters.keras.convert(model, input_names='image', 
                                                image_input_names='image', 
                                                image_scale=1/255.)

0 : input_1, <keras.engine.input_layer.InputLayer object at 0x1a37cc3ba8>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x1a37cc3da0>
2 : re_lu_1, <keras.layers.advanced_activations.ReLU object at 0x1a37cc3fd0>
3 : separable_conv2d_1, <keras.layers.convolutional.SeparableConv2D object at 0x1a37ce9c18>
4 : activation_1, <keras.layers.core.Activation object at 0x1a37ce94e0>
5 : separable_conv2d_2, <keras.layers.convolutional.SeparableConv2D object at 0x1a37d156d8>
6 : add_1, <keras.layers.merge.Add object at 0x10fc69a58>
7 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x1a37d69320>
8 : re_lu_2, <keras.layers.advanced_activations.ReLU object at 0x1a37da4dd8>
9 : separable_conv2d_3, <keras.layers.convolutional.SeparableConv2D object at 0x1a37da44e0>
10 : activation_2, <keras.layers.core.Activation object at 0x1a37dc05f8>
11 : separable_conv2d_4, <keras.layers.convolutional.SeparableConv2D object at 0x1a37dc9160>
12 : add_2, <keras.layers.merge.Add object at 0x1a37df

106 : add_22, <keras.layers.merge.Add object at 0x1a38988588>
107 : re_lu_20, <keras.layers.advanced_activations.ReLU object at 0x1a38988358>
108 : separable_conv2d_39, <keras.layers.convolutional.SeparableConv2D object at 0x1a389a6908>
109 : activation_20, <keras.layers.core.Activation object at 0x1a389c10f0>
110 : separable_conv2d_40, <keras.layers.convolutional.SeparableConv2D object at 0x1a389de5c0>
111 : add_23, <keras.layers.merge.Add object at 0x1a38a0ec50>
112 : conv2d_transpose_4, <keras.layers.convolutional.Conv2DTranspose object at 0x1a38a0e5f8>
113 : add_24, <keras.layers.merge.Add object at 0x1a38a60748>
114 : re_lu_21, <keras.layers.advanced_activations.ReLU object at 0x1a38a606d8>
115 : separable_conv2d_41, <keras.layers.convolutional.SeparableConv2D object at 0x1a38a7c940>
116 : activation_21, <keras.layers.core.Activation object at 0x1a38a98e48>
117 : separable_conv2d_42, <keras.layers.convolutional.SeparableConv2D object at 0x1a38ac0710>
118 : add_25, <keras.layers.me

In [5]:
ml_model.save(mlmodel_path)

In [13]:
import tensorflow as tf 
import numpy as np 

In [ ]:
def residual_block(x, filters=64, kernel_size=(3, 3)):
    shortcut = x
    x = tf.keras.layers.SeparableConv2D(filters / 2, kernel_size, padding='same', depthwise_initializer='he_normal')(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.SeparableConv2D(filters, kernel_size, padding='same', depthwise_initializer='he_normal')(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.Add()([shortcut, x])
    return x


In [ ]:
def BAnet(input_size, train=True, android=False):
    ###########
    # Encoder #
    ###########
    ## 1st line
    
    # input shape -> 1/2
    if android:
        inputs = tf.keras.layers.Input(shape=input_size)
        # for android 
        inputs_s = tf.keras.layers.Lambda(lambda x: x[:, :, :, :3])(inputs)
        conv1 = tf.keras.layers.Conv2D(8, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(inputs_s)
    else:
        inputs = tf.keras.layers.Input(shape=input_size)
        conv1 = tf.keras.layers.Conv2D(8, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(inputs)
    
    # 1/2 -> 1/4
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = residual_block(conv1, filters=64, kernel_size=(3, 3))

    ## 2nd line 1/4 -> 1/8
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv1)
    conv2 = residual_block(conv2, filters=64, kernel_size=(3, 3))

    ## 3rd line 1/8 -> 1/16
    conv3 = tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv2)
    conv3 = residual_block(conv3, filters=64, kernel_size=(3, 3))

    ## 4th line 1/16 -> 1/32
    conv4 = tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv3)
    conv4 = residual_block(conv4, filters=64, kernel_size=(3, 3))

    ###########
    # Decoder #
    ###########
    
    # 1/32 -> 1/16
    ## 4th-inverse line
    conv3_inv = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv4)
    conv3_inv = tf.keras.layers.Add()([conv3, conv3_inv])
    conv3_inv = residual_block(conv3_inv, filters=64, kernel_size=(3, 3))

    # 1/16 -> 1/8
    ## 3rd-inverse line
    conv2_inv = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv4_inv)
    conv2_inv = tf.keras.layers.Add()([conv2, conv2_inv])
    conv2_inv = residual_block(conv2_inv, filters=64, kernel_size=(3, 3))

    # 1/8 -> 1/4
    ## 2nd-inverse line
    conv1_inv = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv3_inv)
    conv1_inv = tf.keras.layers.Add()([conv1, conv1_inv])
    conv1_inv = residual_block(conv1_inv, filters=64, kernel_size=(3, 3))
    
    
    ### for boundary mapping 
    # prejection to 1 channel
    ba_projection = tf.keras.layers.Conv2D(1, (1, 1), padding='same', kernel_initializer='he_normal')(conv1_inv)
    # Upsample to input size
    ba_comparison = tf.keras.layers.Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(ba_projection)
    
    ba_map = tf.keras.layers.Activation("sigmoid")(ba_comparison)
    
    ### Feature Fusion Module
    semantic = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv1_inv)
    semantic = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(semantic)
    
    if android:
        x = tf.keras.layers.Concatenate(axis=-1)([semantic, inputs_s, ba_map])
    else: 
        x = tf.keras.layers.Concatenate(axis=-1)([semantic, inputs, ba_map])
        
    tf.keras.layers.Conv2D(filters=)

    shortcut = x
    
    x = tf.keras.layers.Activation('relu')(x)
    x = SeparableConv2D(3, (3, 3), padding='same', depthwise_initializer='he_normal', name='separable_conv2d_47_')(x)
    x = Activation('relu', name='activation_27')(x)
    x = SeparableConv2D(4, (3, 3), padding='same', depthwise_initializer='he_normal', name='separable_conv2d_48_')(x)
    x = Add(name='add_28')([shortcut, x])
    x = Conv2D(1, (1, 1), name='conv2d_7_')(x)
    out = Activation('sigmoid', name='output')(x)
    
    if train:
        model = Model(inputs=inputs, outputs=[out, ba])

    else :
        model = Model(inputs=inputs, outputs=out)

    return model

In [6]:
def light_matting_net(input_size, train=True, android=False):
    ###########
    # Encoder #
    ###########
    ## 1st line
    if android:
        inputs = Input(input_size)
        # for android 
        inputs_s = Lambda(lambda x: x[:, :, :, :3])(inputs)
        conv1 = Conv2D(8, (3, 3), padding='same', kernel_initializer='he_normal')(inputs_s)
    else:
        inputs = Input(input_size)
        conv1 = Conv2D(8, (3, 3), padding='same', kernel_initializer='he_normal')(inputs)
    
    conv1 = residual_block(conv1, filters=8, kernel_size=(3, 3))

    ## 2nd line
    conv2 = Conv2D(32, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv1)
    conv2 = residual_block(conv2, filters=32, kernel_size=(3, 3))

    ## 3rd line
    conv3 = Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv2)
    conv3 = residual_block(conv3, filters=64, kernel_size=(3, 3))

    ## 4th line
    conv4 = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv3)
    conv4 = residual_block(conv4, filters=128, kernel_size=(3, 3))

    ## 5th line
    conv5 = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv4)
    conv5 = residual_block(conv5, filters=128, kernel_size=(3, 3))
    
    ###########
    # Decoder #
    ###########
    ## 4th-inverse line
    conv4_inv = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv5)
    conv4_inv = add([conv4, conv4_inv])
    conv4_inv = residual_block(conv4_inv, filters=128, kernel_size=(3, 3))

    ## 3rd-inverse line
    conv3_inv = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv4_inv)
    conv3_inv = add([conv3, conv3_inv])
    conv3_inv = residual_block(conv3_inv, filters=64, kernel_size=(3, 3))
    
    ## 2nd-inverse line
    conv2_inv = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv3_inv)
    conv2_inv = add([conv2, conv2_inv])
    conv2_inv = residual_block(conv2_inv, filters=32, kernel_size=(3, 3))

    ## 1st-inverse line
    conv1_inv = Conv2DTranspose(8, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv2_inv)
    conv1_inv = add([conv1, conv1_inv])
    conv1_inv = residual_block(conv1_inv, filters=8, kernel_size=(3, 3))
    
    ### Boundary attention map 추가
    b = Conv2D(1, (1, 1), padding='same', kernel_initializer='he_normal', name="boundary_conv1d_")(conv1_inv)
    ba = Activation("sigmoid", name='boundary_attention')(b)
    
    conv6 = Conv2D(3, (1, 1))(conv1_inv)
    x = Activation('tanh')(conv6)
    
    # Concatenating
    x = Concatenate(axis=-1)([x, ba])

    shortcut = x
    
    x = ReLU(name='re_lu_24')(x)
    x = SeparableConv2D(3, (3, 3), padding='same', depthwise_initializer='he_normal', name='separable_conv2d_47_')(x)
    x = Activation('relu', name='activation_27')(x)
    x = SeparableConv2D(4, (3, 3), padding='same', depthwise_initializer='he_normal', name='separable_conv2d_48_')(x)
    x = Add(name='add_28')([shortcut, x])
    x = Conv2D(1, (1, 1), name='conv2d_7_')(x)
    out = Activation('sigmoid', name='output')(x)
    
    if train:
        model = Model(inputs=inputs, outputs=[out, ba])

    else :
        model = Model(inputs=inputs, outputs=out)

    return model



In [1]:
from models import * 

model = light_matting_net(input_size=(512, 512, 3))

Using TensorFlow backend.
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 256, 256, 8)  144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           separable_conv2d_1[0][0]         
____________________________________________________________________________________________